<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Заполнение-пропусков" data-toc-modified-id="Заполнение-пропусков-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Заполнение пропусков</a></span></li><li><span><a href="#Проверка-данных-на-аномалии-и-исправления" data-toc-modified-id="Проверка-данных-на-аномалии-и-исправления-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка данных на аномалии и исправления</a></span></li><li><span><a href="#Изменение-типов-данных" data-toc-modified-id="Изменение-типов-данных-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Изменение типов данных</a></span></li><li><span><a href="#Удаление-дубликатов" data-toc-modified-id="Удаление-дубликатов-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Удаление дубликатов</a></span></li><li><span><a href="#Формирование-дополнительных-датафреймов-словарей,-декомпозиция-исходного-датафрейма." data-toc-modified-id="Формирование-дополнительных-датафреймов-словарей,-декомпозиция-исходного-датафрейма.-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.</a></span></li><li><span><a href="#Категоризация-дохода" data-toc-modified-id="Категоризация-дохода-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Категоризация дохода</a></span></li><li><span><a href="#Категоризация-целей-кредита" data-toc-modified-id="Категоризация-целей-кредита-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Категоризация целей кредита</a></span></li><li><span><a href="#Ответы-на-вопросы" data-toc-modified-id="Ответы-на-вопросы-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Ответы на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надежности заёмщиков

Заказчик - кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. 

Данные от банка - статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга - специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

**Импортируем библиотеку Pandas и прочитаем файл с данными о клиентах банка. Выведем на экран первые 10 строк таблицы для ознакомления.**

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('D:/datasets/data.csv')
except:
    data = pd.read_csv('/datasets/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Посмотрим общую информацию о таблице.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


* В таблице 12 столбцов, 21 525 строк. Типы данных float64, int64, object. 

Данные в таблице:

* `children` — количество детей в семье;

* `days_employed` — общий трудовой стаж в днях;

* `dob_years` — возраст клиента в годах;

* `education` — уровень образования клиента;

* `education_id` — идентификатор уровня образования;

* `family_status` — семейное положение;

* `family_status_id` — идентификатор семейного положения;

* `gender` — пол клиента;

* `income_type` — тип занятости;

* `debt` — имел ли задолженность по возврату кредитов;

* `total_income` — ежемесячный доход;

* `purpose` — цель получения кредита.

In [4]:
# проверим названия столбцов
data.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

**Названия столбцов соответствуют хорошему стилю:**

* написаны в «змеином_регистре»;
* все символы строчные;
* пробелы отсутствуют.

Для удобства переименуем столбец `dob_years` - возраст клиента, на `age`.

In [5]:
data = data.rename(columns={'dob_years': 'age'})

## Заполнение пропусков

Для начала посчитаем сколько всего пропусков в таблице.

In [6]:
data.isna().sum()

children               0
days_employed       2174
age                    0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

* В столбцах `days_employed` - общий трудовой стаж и `total_income` - ежемесячный доход обнаружено по 2174 пропущенных значения NaN, эти данные важны для нашего исследования, поэтому удалять их мы не будем, а заменим их на медианные и средние значения. 

**Посчитаем какую долю составляют пропущенные значения в каждом из столбцов с пропусками.**

In [7]:
print('Доля пропусков в столбце трудовой стаж - {:.0%}'.format(data['days_employed'].isna().sum() / len(data)))


Доля пропусков в столбце трудовой стаж - 10%


In [8]:
print('Доля пропусков в столбце ежемесячный доход - {:.0%}'.format(data['total_income'].isna().sum() / len(data)))

Доля пропусков в столбце ежемесячный доход - 10%


**Cформируем сводную таблицу с группировкой по возрасту и измерением среднего трудового стажа.**

In [9]:
group_employed = data.groupby('age')['days_employed'].transform('mean')

**Заполним пропуски в столбце 'days_employed' по каждой возрастной группе на среднее.**

In [10]:
data['days_employed'] = data['days_employed'].fillna(group_employed)

**Cформируем сводную таблицу с группировкой по типу занятости и измерением медианного дохода.**

In [11]:
group_income = data.groupby('income_type')['total_income'].transform('median')

**Теперь заменим пропуски медианой по каждой группе и округлим значения до 2 знаков после запятой.**

In [12]:
data['total_income'] = round(data['total_income'].fillna(group_income), 2)

In [13]:
data.isna().sum()

children            0
days_employed       0
age                 0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

## Проверка данных на аномалии и исправления

**Проверим данные с помощью метода describe()**

In [14]:
data.describe().T.round(2)

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.54,1.38,-1.00,0.00,0.00,1.00,20.00
days_employed,21525.0,63311.24,137241.21,-18388.95,-2518.17,-1022.59,-115.37,401755.40
age,21525.0,43.29,12.57,0.00,33.00,42.00,53.00,75.00
education_id,21525.0,0.82,0.55,0.00,1.00,1.00,1.00,4.00
family_status_id,21525.0,0.97,1.42,0.00,0.00,0.00,1.00,4.00
debt,21525.0,0.08,0.27,0.00,0.00,0.00,0.00,1.00
total_income,21525.0,165225.32,98043.67,20667.26,107798.17,142594.40,195549.94,2265604.03


* Столбец `days_employed` и `children` имеет отрицательные значения.


* В столбце `children` присутствует аномально высокое значение 20.


* В столбце `days_employed` аномально высокое максимальное значение трудового стажа.


* Минимальный возраст клиента, согласно данным - 0.


**Исправим аномалии:**


* С помощью метода abs(), приведём значения в столбцах с количеством трудового стажа и количеством детей к положительным.

In [15]:
data['days_employed'] = abs(round(data['days_employed'], 1))
data['children'] = abs(data['children'])

* Максимальное количество детей - 20, вероятно это опечатка, найдем медиану и заменим.

In [16]:
children_median = data['children'].median()
data['children'] = data['children'].replace(20, children_median)
data['children'].max()

5.0

Теперь максимальное значение по столбцу 5.

* В столбце `days_employed` есть аномально высокие значения трудового стажа.

In [17]:
data.groupby('income_type')['days_employed'].mean()

income_type
безработный        366413.650000
в декрете            3296.800000
госслужащий          6976.176354
компаньон            5043.579233
пенсионер          348237.245332
предприниматель     92956.000000
сотрудник            5310.743160
студент               578.800000
Name: days_employed, dtype: float64

* В категориях пенсионеры, безработные и предприниматели очень высокие значения. Заменим эти значения на средний стаж работы, по России, это 34 года, 12 410 дней, если трудовой стаж больше чем возраст клиента.

In [18]:
days_employed_mean = 12410
data.loc[(data['days_employed'] > (data['age'] * 365)), 'days_employed'] = days_employed_mean
data['days_employed'].max()

18388.9

Теперь максимальное значение по столбцу 18388 дней, это 50 лет.


* Проверим значения столбца `age`, согласно данным таблицы минимальный возраст клиента - 0 лет, что невозможно для заёмщика. Рассчитаем и заменим на медианные значения по семейному статусу (кажется эти признаки вполне коррелируют).

In [19]:
group_age_median = data.groupby('family_status')['age'].transform('median')

In [20]:
data.loc[(data['age'] == 0), 'age'] = group_age_median
data['age'].min()

19

Минимальный возраст клиентов после замены 19 лет.

**Проверим уникальные значения в столбце `gender`.**

In [21]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

* Обнаружено одно некорректное значение `XNA`. Заменим на наиболее популярное значение пола в таблице. В данном случае это женский пол - Female.

In [22]:
data['gender'] = data['gender'].replace('XNA', 'F')

**Выводы:**

## Изменение типов данных

**Заменим вещественный тип данных в столбцах `total_income` и `days_employed` на целочисленный и проверим.**

In [23]:
data['total_income'] = data[['total_income', 'days_employed']].astype('int')
data[['total_income', 'days_employed']].dtypes

total_income       int32
days_employed    float64
dtype: object

## Удаление дубликатов

**Посчитаем сколько дубликатов в таблице.**

In [24]:
data.duplicated().sum()

54

**Проверим уникальные значения в столбце `education`.**

In [25]:
data['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

**Приведём с помощью метода str.lower() все буквы к нижнему регистру.**

In [26]:
data['education'] = data['education'].str.lower()
data['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

**Проверим оставшиеся столбцы.**

In [27]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

* В столбце цель кредита много неявнях дубликатов, позднее проведём категоризацию и объединим цели.

In [28]:
data['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [29]:
data['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

**Удалим дубликаты и восстановим индексацию.**

In [30]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

**Создадим два новых датафрейма со столбцами:**

* `education_id` и `education` — в первом;


* `family_status_id` и `family_status` — во втором.

In [31]:
education_data = data[['education_id', 'education']]
education_data.head()

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее


In [32]:
family_data = data[['family_status_id', 'family_status']]
family_data.head()

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак


**Удалим дубликаты и восстановим индексы.**

In [33]:
education_data = education_data.drop_duplicates().reset_index(drop=True)
family_data = family_data.drop_duplicates().reset_index(drop=True)

* Из исходного датафрейма удалим столбцы `education_id` и `family_status_id`.

In [34]:
data.drop(['education_id', 'family_status_id'], axis=1, inplace=True)
data.head()

,children,days_employed,age,education,family_status,gender,income_type,debt,total_income,purpose
0,1.0,8437.7,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья
1,1.0,4024.8,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623.4,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья
3,3.0,4124.7,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование
4,0.0,12410.0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу


## Категоризация дохода

**Создадим функцию, которая на основании доходов присвоит клиентам определённую категорию и проверим её работу.**

In [35]:
def total_income_category(total_income):
    """
    Возвращает категорию исходя из суммы дохода total_income, используя правила:
    - 'E', если total_income от 0 до 30000;
    - 'D', если total_income от 30001 до 50000;
    - 'С', если total_income от 50001 до 200000;
    - 'B', если total income от 200001 до 1000000;
    - 'A', если total_income 1000001 и выше.
    """
    
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'

print(total_income_category(29000))
print(total_income_category(45000))
print(total_income_category(78000))
print(total_income_category(250000))
print(total_income_category(1000250))                           

E
D
C
B
A


* Написанная функция работает корректно, теперь создадим столбец total_income_category, в который на основании дохода будет записана категория. 

In [36]:
data['total_income_category'] = data['total_income'].apply(total_income_category)
data.head()

,children,days_employed,age,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,8437.7,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,B
1,1.0,4024.8,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,5623.4,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,C
3,3.0,4124.7,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,B
4,0.0,12410.0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,C


## Категоризация целей кредита

**Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

* `операции с автомобилем`;
* `операции с недвижимостью`;
* `проведение свадьбы`;
* `получение образования`.

In [37]:
def purpose_category(purpose):
        if 'свадьб' in purpose:
            return 'проведение свадьбы'
        elif 'автомобил' in purpose:
            return 'операции с автомобилем'
        elif 'образован' in purpose:
            return 'получение образования'
        elif 'жиль' or 'недвижимост' in purpose:
            return 'операции с недвижимостью'
        
data['purpose_category'] = data['purpose'].apply(purpose_category)
data.head()        

,children,days_employed,age,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1.0,8437.7,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1.0,4024.8,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0.0,5623.4,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3.0,4124.7,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0.0,12410.0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


## Ответы на вопросы

**Проверим есть ли зависимость между количеством детей и возвратом кредита в срок.**

* Для начала разделим клиентов на три категории, в зависимости от количества детей.

In [38]:
def children_category(children):
    if children == 0:
        return 'нет детей'
    elif children <= 3:
        return 'малодетная семья'
    elif children > 3:
        return 'многодетная семья'
    
data['children_category'] = data['children'].apply(children_category)
data.head()

,children,days_employed,age,education,family_status,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1.0,8437.7,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,малодетная семья
1,1.0,4024.8,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,малодетная семья
2,0.0,5623.4,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3.0,4124.7,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,B,получение образования,малодетная семья
4,0.0,12410.0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей


* Создадим сводные таблицы, сгруппируем данные по столбцам `children_category` и `debt`.

In [39]:
data_pivot = data.pivot_table(index=['children_category'], values='debt', aggfunc=['mean'])
data_pivot

,mean
,debt
children_category,
малодетная семья,0.092027
многодетная семья,0.080000
нет детей,0.075598


* Клиенты, у которых 1-2 ребенка, больше просроченных задолженностей, чем у клиентов с 3 и более детьми. Клиенты у которых нет детей имеют меньше просроченной задолженности. Данные не однозначные, возможно, нужна выборка больше.

**Проверим есть ли зависимость между семейным положением и возвратом кредита в срок.**

In [40]:
data_pivot = data.pivot_table(index=['family_status'], values='debt', aggfunc=['mean'])
data_pivot

,mean
,debt
family_status,
Не женат / не замужем,0.097509
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452


* Как можно заметить, статус официальности отношений действительно положительно сказывается на возврате кредита. Люди, которые не состоят в официальных отношениях и никогда не состояли хуже возвращают кредиты, в отличии от семей, которые имеют официальный статус, пусть даже когда-то в прошлом.    

**Проверим есть ли зависимость между уровнем дохода и возвратом кредита в срок.**

Разделим уровень доходов клиентов на диапазоны с учетом квартилей распределения. Квартили распределения делятся на 25%. Иными словами, все что до 1 квартиля (25%) — низкий доход, между 1 и медианой(2 квартиль) тоже будет 25% — средний доход, от медианы до 3 квартиля (25%) — выше среднего, а все что дальше 3 квартиля — высокий доход. Воспользуемся функцией pd.qcut, которая определяет интервалы с использованием процентилей на основе распределения данных, а не фактических числовых границ интервалов.

In [41]:
data['quantile_total_income'] = pd.qcut(data['total_income'], q=4, labels=['низкий доход', 
                                                                                      'средний доход',
                                                                                      'доход выше среднего', 
                                                                                      'высокий доход'])

In [42]:
data.pivot_table(index=['quantile_total_income'], values='debt', aggfunc=['mean'])

,mean
,debt
quantile_total_income,
низкий доход,0.079605
средний доход,0.088155
доход выше среднего,0.085382
высокий доход,0.071402


* Как видно, имеется некоторая зависимость, клиенты со средним доходом чаще имеют просроченную задолженность.

Клиенты с низким доходом вероятнее всего просто не берут кредит на то, чего не смогут вернуть, проходят куда более строгий контроль. А клиенты с высоким достатком просто обладают достаточными средствами на погашение. Клиенты со средними доходами вероятно слишком оптимистично подходят к своим финансовыми возможностям.

**Проверим как разные цели кредита влияют на его возврат в срок.**

In [43]:
data_pivot = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['mean'])
data_pivot

,mean
,debt
purpose_category,
операции с автомобилем,0.093590
операции с недвижимостью,0.072334
получение образования,0.092200
проведение свадьбы,0.080034


Среди 4 категорий, чаще всего платят в срок за кредит на недвижимость, а просрочка чаще возникает на кредит на автомобиль и образование. Кредиты на свадьбу имеют меньше просроченных платежей.


* Недвижимость как правило находится в залоге у банка, к тому же люди осознанно подходят к взятию кредита на данную цель.


* Образование вероятно не сразу окупается, так как студент должен набраться опыта. 


* Автомобиль влечет за собой множество дополнительных расходов, о которых люди часто забывают, когда берут на него кредит.


* Кредит на свадьбу вероятно погашается с подарков :)

## Общий вывод

* Итак, семейный статус влияет на вероятность платежей по кредиту в срок.


* Люди, состоящие в браке чаще плятят в срок чем те, кто не был в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. 


* Чем больше детей, тем больше просрочек по кредиту, люди не имеющие детей реже допускают просроченныую задолженность.


* Цель кредита так же влияет на просроченные платежи, за кредит на недвижимость просреченные платежи возникают реже, просрочка чаще возникает на кредит на автомобиль и образование. Кредиты на свадьбу имеют меньше просроченных платежей.